# 1. Given optimal keffs, choose fewest keff changes--i.e., most sensitive ones
Two-step procedure:
1. Use favorite method to get optimal keffs
1. Run MILP to choose the fewest keffs differing from nominal keffs

$$
\begin{align*}
\min_{v,y,z} \quad & \sum_k \| (c^k)^T v^k - \mu^{*k} \| \\
\mathrm{s.t.} \quad & S^k v^k = 0 \\
& \sum_i x_i^k \leq c^k \\
& x_i^k = \alpha_i v_i^k + \beta_i z_i^k - \alpha_i z_i^k \\
& -M\cdot (1-y_i) \leq z_i^k - v_i^k \leq M \cdot (1-y_i) \\
& l_i^k y_i \leq z_i^k \leq u_i^k y_i \\
& l_i^k \leq v_i^k \leq u_i^k \\
& \sum_i y_i \leq K \\
& [\text{Duality gap constraint}] \\
& y_i \in \{0,1\}
\end{align*}
$$

With duality gap constraints:
$$
\begin{align*}
\min_{v,y,z} \quad & \sum_i y_i\\
\mathrm{s.t.} \quad & S^k v^k = 0 \\
& sp^k - sn^k = (c^k)^T v^k - \mu^{*k} \\
& \sum_i x_i^k \leq C^k \\
& x_i^k = \alpha_i v_i^k + \beta_i z_i^k - \alpha_i z_i^k \\
& -M\cdot (1-y_i) \leq z_i^k - v_i^k \leq M \cdot (1-y_i) \\
& l_i^k y_i \leq z_i^k \leq u_i^k y_i \\
& l_i^k \leq v_i^k \leq u_i^k \\
& sp^k + sn^k \leq f^k \\
& (c^k)^T v^k = w^a C + w^u u - w^l l \\
& w^S S + w^u - w^l + w^a \alpha_i + z^a_i (\beta_i - \alpha_i)  = (c^k)^T \\
& -M\cdot (1-y_i) \leq z_i^{a,k} - w^{a,k} \leq M \cdot (1-y_i) \\
& w^{a,l} y_i \leq z_i^{a,k} \leq w^{a,u} y_i \\
& y_i \in \{0,1\} \\
& sp, sn \geq 0 \\
& w^u, w^l, w^a \geq 0 \\
& v^k, w^s, z \in \mathbb{R}
\end{align*}
$$

## MILP tips
- Callbacks via [Python API of Gurobi](http://www.gurobi.com/documentation/7.5/refman/py_python_api_overview.html#sec:Python)
    - Lazy constraints (e.g., for Benders decomposition): [Model.cbLazy](http://www.gurobi.com/documentation/7.5/refman/py_model_cblazy.html#pythonmethod:Model.cbLazy)
    - User cuts: [Model.cbCut](http://www.gurobi.com/documentation/7.5/refman/py_model_cbcut.html#pythonmethod:Model.cbCut)
- Eliminate incumbents via tighter McCormick relaxation of bilinear constraints
    - 20% speedup of radix method

Alternatively, run one-shot MINLP
- could reformulate to gigantic MILP, e.g., via radix reformulation

In [336]:
from cobra import Metabolite, Reaction
from dynamicme.optimize import Constraint, Variable

In [154]:
from cobra.io import load_json_model

ijofit = load_json_model('/home/laurence/ME/data/dynamicME/best_ijomc.json')

In [341]:
ijomc = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc_cons.json')

## Simple one condition problem

In [388]:
F_ERROR = 0.05
M = 1e4

### Create basic MILP using cobrapy
#ijomilp    = load_json_model('/home/laurence/ME/models/BiGG_M/json/iJO1366.json')
ijomilp = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc_cons.json')

crowding0 = ijomc.metabolites.crowding
cons_crowd = Constraint('crowding')
cons_crowd._constraint_sense = 'L'
cons_crowd._bound = crowding0._bound
ijomilp.add_metabolites([cons_crowd])
### mu = wS*b + wa*C + wu*u - wl*l
### mu - wS*b - wa*C - wu*u + wl*l = 0
cons_gap = Constraint('cons_duality_gap')
ijomilp.add_metabolites([cons_gap])
xs = []
ys = []
zs = []
for rxn0 in crowding0.reactions:
    ### Add sum_i xi <= c
    x = Variable('x_'+rxn0.id)
    xs.append(x)
    x.add_metabolites({cons_crowd:1})
    ### Add binary constraints
    ### xki = αi vki + βi zki − αi zki
    ### --> xki - ai vki - bi zki + ai zki = 0
    cons_x = Constraint('cons_'+x.id)
    x.add_metabolites({cons_x:1.})
    ai = rxn0.metabolites[crowding0]
    rxn_fit = ijofit.reactions.get_by_id(rxn0.id)
    crowd_fit = ijofit.metabolites.crowding
    bi = rxn_fit.metabolites[crowd_fit]
    vi = ijomilp.reactions.get_by_id(rxn0.id)
    vi.add_metabolites({cons_x:-ai})
    zi = Variable('z_'+rxn0.id)
    zs.append(zi)
    zi.add_metabolites({cons_x:-bi + ai})
    ### -M*(1-yi) <= zi - vi <= M*(1-yi)
    yi = Variable('y_'+rxn0.id)
    ys.append(yi)
    yi.variable_kind = 'integer'
    yi.lower_bound = 0.
    yi.upper_bound = 1.
    cons_yl = Constraint('cons_z_diff_L_'+rxn0.id)    
    cons_yl._constraint_sense = 'L'
    cons_yl._bound = M
    yi.add_metabolites({cons_yl:M})
    zi.add_metabolites({cons_yl:-1.})
    vi.add_metabolites({cons_yl:1.})
    cons_yu = Constraint('cons_z_diff_U_'+rxn0.id)
    cons_yu._constraint_sense = 'L'
    cons_yu._bound = M
    yi.add_metabolites({cons_yu:M})
    zi.add_metabolites({cons_yu:1.})
    vi.add_metabolites({cons_yu:-1.})
    ### l*yi <= zi <= u*yi
    cons_zl = Constraint('cons_z_L_'+rxn0.id)
    cons_zu = Constraint('cons_z_U_'+rxn0.id)
    cons_zl._constraint_sense = 'L'
    cons_zu._constraint_sense = 'L'
    zi.add_metabolites({cons_zl:-1.})
    yi.add_metabolites({cons_zl:rxn.lower_bound})
    zi.add_metabolites({cons_zu:1.})
    yi.add_metabolites({cons_zu:-rxn.upper_bound})
    
ijomilp.add_reactions(xs)
ijomilp.add_reactions(ys)
ijomilp.add_reactions(zs)

### L1 norm objective 
for rxn in ijomilp.reactions:
    rxn.objective_coefficient = 0.

for y in ys:
    y.objective_coefficient = 1.
    
mu_p = Variable('mu_p')
mu_n = Variable('mu_n')
ijomilp.add_reactions([mu_p, mu_n])

mu_p.lower_bound = 0.
mu_n.lower_bound = 0.
#mu_p.objective_coefficient = 1.
#mu_n.objective_coefficient = 1.
### sp−sn - c'vk = -μ∗
cons_norm = Constraint('cons_norm')
mu_exp = 0.740
cons_norm._bound = -mu_exp
mu_p.add_metabolites({cons_norm:1.})
mu_n.add_metabolites({cons_norm:-1.})
rxn_obj = ijomilp.reactions.BIOMASS_Ec_iJO1366_core_53p95M
rxn_obj.add_metabolites({cons_norm:-1.})

### sp + sn <= f
cons_obj = Constraint('cons_obj')
cons_obj._constraint_sense = 'L'
cons_obj._bound = F_ERROR*mu_exp   # 1% error allowed
mu_p.add_metabolites({cons_obj:1.})
mu_n.add_metabolites({cons_obj:1.})

### Duality gap constraints
wSs = []
wus = []
wls = []
wa  = []
zas = []
waL = 0.
waU = 1e6

rxn_obj.add_metabolites({cons_gap:1.})

wa = Variable('wa')
wa.lower_bound = 0.
ijomilp.add_reaction(wa)

wa.add_metabolites({cons_gap:-cons_crowd._bound})

for met in ijomilp.metabolites:
    if not isinstance(met, Constraint):
        wSi = Variable('wS_'+met.id)
        if met._constraint_sense == 'E':
            wSi.lower_bound = -M
        else:
            wSi.lower_bound = 0.        
        if met._constraint_sense == 'G':
            wSi.add_metabolites({cons_gap:-met._bound})
        else:
            wSi.add_metabolites({cons_gap:met._bound})
        wSs.append(wSi)
    
ijomilp.add_reactions(wSs)

for rxn in ijomilp.reactions:
    if not isinstance(rxn, Variable):
        wui = Variable('wu_'+rxn.id)
        wli = Variable('wl_'+rxn.id)
        wui.lower_bound = 0.
        wli.lower_bound = 0.
        wus.append(wui)
        wls.append(wli)
        wui.add_metabolites({cons_gap:-rxn.upper_bound})
        wli.add_metabolites({cons_gap:rxn.lower_bound})
        ### wS*S + wu - wl + wa*a + za*(b-a) = c
        cons_dual = Constraint('cons_dual_'+rxn.id)
        cons_dual._constraint_sense = 'E' # since v can be neg                
        rxn0 = ijomc.reactions.get_by_id(rxn.id)
        cons_dual._bound = rxn0.objective_coefficient
        # Add the wS*S terms
        for met,si in rxn.metabolites.iteritems():
            if not isinstance(met, Constraint):        
                wSi = ijomilp.reactions.get_by_id('wS_'+met.id)
                if met._constraint_sense == 'G':                    
                    wSi.add_metabolites({cons_dual:-si})
                else:                    
                    wSi.add_metabolites({cons_dual:si})
        wui.add_metabolites({cons_dual:1.})
        wli.add_metabolites({cons_dual:-1.})

for rxn in ijomilp.metabolites.crowding.reactions:
    if not isinstance(rxn,Variable):
        # if this rxn participates in crowding:
        #if rxn.metabolites.has_key(cons_crowd):
        cons_dual = ijomilp.metabolites.get_by_id('cons_dual_'+rxn.id)
        rxn0 = ijomc.reactions.get_by_id(rxn.id)
        ai = rxn0.metabolites[crowding0]
        rxn_fit = ijofit.reactions.get_by_id(rxn.id)
        bi = rxn_fit.metabolites[ijofit.metabolites.crowding]
        wa.add_metabolites({cons_dual:ai})
        zai = Variable('za_'+rxn.id)
        zai.lower_bound = 0.
        zas.append(zai)
        zai.add_metabolites({cons_dual:bi-ai})
        # -M*(1-yi) <= wa - zai <= M*(1-yi)
        yi = ijomilp.reactions.get_by_id('y_'+rxn.id)
        cons_zaL = Constraint('cons_za_diff_L_'+rxn.id)
        cons_zaU = Constraint('cons_za_diff_U_'+rxn.id)
        cons_zaL._constraint_sense = 'L'
        cons_zaU._constraint_sense = 'L'
        cons_zaL._bound = M
        cons_zaU._bound = M
        zai.add_metabolites({cons_zaL:1.})
        wa.add_metabolites({cons_zaL:-1.})
        yi.add_metabolites({cons_zaL:M})
        zai.add_metabolites({cons_zaU:-1.})
        wa.add_metabolites({cons_zaU:1.})
        yi.add_metabolites({cons_zaU:M})
        # wal*yi <= zai <= wau*yi
        cons_zaL = Constraint('cons_za_L_'+rxn.id)
        cons_zaU = Constraint('cons_za_U_'+rxn.id)
        cons_zaL._constraint_sense = 'L'
        cons_zaU._constraint_sense = 'L'
        zai.add_metabolites({cons_zaL:-1.})
        yi.add_metabolites({cons_zaL:waL})
        zai.add_metabolites({cons_zaU:1.})
        yi.add_metabolites({cons_zaU:-waU})

ijomilp.add_reactions(wus)
ijomilp.add_reactions(wls)
ijomilp.add_reactions(zas)

### Ungroup Sv=0 with these other ineqs for dual cons

In [389]:
print len(ijomc.metabolites), len(ijomc.reactions)
print len(xs), len(ys), len(zs)
print len(zas), len(wls), len(wus), len(wSs)

2972 3166
2018 2018 2018
2018 3166 3166 2972


In [393]:
F_ERROR = 0.3
cons_obj._bound = F_ERROR*mu_exp   # 1% error allowed

In [394]:
%%time
ijomilp.optimize(solver='gurobi', objective_sense='minimize')

CPU times: user 7min 19s, sys: 5.44 s, total: 7min 24s
Wall time: 1min 53s


<Solution 30.00 at 0x7f7d87715550>

In [395]:
ijomilp.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.5179999999971869

In [396]:
ijomilp.reactions.y_SUCDi.reaction

'1000.0 cons_z_U_SUCDi + 1000000.0 cons_za_U_SUCDi --> 10000.0 cons_z_diff_L_SUCDi + 10000.0 cons_z_diff_U_SUCDi + 10000.0 cons_za_diff_L_SUCDi + 10000.0 cons_za_diff_U_SUCDi'

## Check by plugging in opt keffs back into primal

In [397]:
ijomc0 = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc_cons.json')

print ijomc0.optimize()

for rxn0 in ijomc0.metabolites.crowding.reactions:
    yi = ijomilp.reactions.get_by_id('y_'+rxn0.id)
    if yi.x > 0.99:
        rxnfit = ijofit.reactions.get_by_id(rxn0.id)
        bi = rxnfit.metabolites[ijofit.metabolites.crowding]
        ai = rxn0.metabolites[ijomc0.metabolites.crowding]
        print 'Changing keffo of %s from ai=%g to bi=%g'%(rxn0.id, ai, bi)
        rxn0._metabolites[ijomc0.metabolites.crowding] = bi

<Solution 0.74 at 0x7f7d82afb450>
Changing keffo of ATPPRT from ai=0.000586461 to bi=6.83214e-07
Changing keffo of BPNT from ai=0.000405287 to bi=3.87926e-06
Changing keffo of GAPD_abs from ai=3.72556e-06 to bi=2.30943e-09
Changing keffo of IG3PS from ai=0.00062262 to bi=7.86317e-05
Changing keffo of H2Otex_abs from ai=1.68237e-05 to bi=1.13398e-06
Changing keffo of QULNS from ai=0.0277778 to bi=8.96067e-06
Changing keffo of HCO3E_abs from ai=2.00027e-05 to bi=4.32967e-09
Changing keffo of CO2tex_abs from ai=1.33094e-05 to bi=4.35725e-06
Changing keffo of CO2tpp_abs from ai=1.33547e-05 to bi=7.28495e-08
Changing keffo of O2tpp_abs from ai=1.33547e-05 to bi=2.2427e-06
Changing keffo of CDPMEK from ai=0.0277778 to bi=0.00174286
Changing keffo of ACGK from ai=0.000455324 to bi=8.69941e-05
Changing keffo of ADK1_abs from ai=3.85861e-05 to bi=5.96668e-06
Changing keffo of NADS1 from ai=0.0277778 to bi=2.42183e-05
Changing keffo of EDA from ai=5.65228e-06 to bi=1.18587e-06
Changing keffo of 

In [398]:
print ijomc0.optimize()

<Solution 0.98 at 0x7f7d82a7e9d0>


In [399]:
cons_errs = []
for rxn in cons_gap.reactions:
    cons_errs.append(rxn.x * rxn.metabolites[cons_gap])
sum(cons_errs)

-1.7763568394002505e-15

In [400]:
sum([y.x for y in ys])

30.0

In [ ]:
model.vari

In [359]:
for i,z in enumerate(ijomilp.reactions.query('z_')):
    if abs(z.x) > 1e-4:
        #yi = ijomilp.reactions.get_by_id('y_'+)
        print z, '\t', z.x

z_H2Otpp_abs 	18.4977141594
wS_4abz_c 	999.273372851
wS_4hbz_c 	999.360309453
wS_4mhetz_c 	985.521705617
wS_4mpetz_c 	985.41307122
wS_5caiz_c 	-0.817685480274
wS_5prdmbz_c 	-1000.0
wS_dmlz_c 	478.783149922
wS_sucbz_c 	999.200879397


In [228]:
# ijomilp.metabolites.cons_duality_gap.reactions

In [285]:
from cobra.solvers import gurobi_solver

model = gurobi_solver.create_problem(ijomilp)

In [286]:
model.Params.IntFeasTol

1e-09

In [287]:
model.Params.MIPFocus = 2 # prove optimal

In [311]:
from gurobipy import *

model.ModelSense = GRB.MINIMIZE

In [312]:
model.optimize()

In [313]:
sol = gurobi_solver.format_solution(model, ijomilp)

In [314]:
var_dict = {r.id:x for r,x in zip(ijomilp.reactions, model.getVars())}

In [315]:
xx = var_dict['y_FE3Ri']
xx.Obj

1.0

In [316]:
from six import iteritems
import re

yids = []
for v,x in iteritems(sol.x_dict):
    if re.match(r"^y_", v):
        yids.append(v)
        if abs(x)>1e-4:
            print x, '\t', v

In [317]:
len(yids)

2018

In [318]:
sol.x_dict[ijomilp.reactions.BIOMASS_Ec_iJO1366_core_53p95M.id]

0.7029999999971324

In [21]:
from gurobipy import *

#GRB.Callback.callback()

<bound method CallbackClass.callback of <gurobipy.CallbackClass object at 0x7f7e08a58390>>